In [1]:
import glob
import os
import fitz
import re

from RFB_QnA_parsing import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATA_PATH="data/"

In [4]:
glob.glob(DATA_PATH + "*")

['data/Scraping Acórdãos-20250111T184338Z-023.zip',
 'data/pr-irpf-2019.pdf',
 'data/normas_full.json',
 'data/pr-irpf-2016.pdf',
 'data/extracted_questions_20250202_parcial_02.pkl',
 'data/extracted_questions_20250202_parcial_01.pkl',
 'data/pr-irpf-2017.pdf',
 'data/pr-irpf-2021.pdf',
 'data/extracted_questions_20250203_parcial_01.pkl',
 'data/extracted_questions_20250202_parcial_03.pkl',
 'data/llama_deduplicated_reference_titles_20250205_02.pkl',
 'data/chunks_normas_new.pkl',
 'data/P&R IRPF 2023 - v1.1 - 04042023.pdf',
 'data/extracted_questions_20241029.pkl',
 'data/pr-irpf-2020.pdf',
 'data/pr-irpf-2018.pdf',
 'data/Scraping Normas-20250111T184306Z-001.zip',
 'data/PR-IRPF-2022-v-1-2-2022-05-16.pdf',
 'data/P&R IRPF 2024 - v1.0 - 2024.05.03.pdf',
 'data/extracted_questions_20250203_final.pkl']

In [6]:
questions = extract_answers(DATA_PATH + "P&R IRPF 2024 - v1.0 - 2024.05.03.pdf")

Processing file data/P&R IRPF 2024 - v1.0 - 2024.05.03.pdf with 301 pages...



{'question_number': '001', 'question_summary': 'OBRIGATORIEDADE', 'question_text': 'Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024,', 'answer': []}


Começo pergunta. questão=001
Achou fim pergunta. questão=001
[]
Achou fim. questão=001. Total=1


{'question_number': '002', 'question_summary': 'PESSOA FÍSICA DESOBRIGADA', 'question_text': 'Pessoa física desobrigada pode apresentar a Declaração de Ajuste Anual (DAA)?', 'answer': []}


Começo pergunta. questão=002
['Consulte as perguntas 001 e 174']
Achou fim. questão=002. Total=2


{'question_number': '003', 'question_summary': 'TITULAR OU SÓCIO DE EMPRESA', 'question_text': 'Contribuinte que é titular ou sócio de empresa está obrigado a apresentar a Declaração de', 'answer': []}


Começo pergunta. questão=003
Achou fim pergunta. questão=003
['Consulte a pergunta 001']
Achou fim. questão=003. Total=3


{'question_n

In [7]:
print(len(questions))

715


## Check for missing or wrongly numbered questions

In [8]:
last_question = 0

for which_question in questions:
    print(which_question['question_number'])

    if last_question + 1 != int(which_question['question_number']):
        print(f"Missing questions: last_question={last_question}, current_question={int(which_question['question_number'])}")

    last_question = int(which_question['question_number'])

001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
0820
Missing questions: last_question=81, current_question=820
083
Missing questions: last_question=820, current_question=83
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221

#### Fix the only wrong question number ― problem in the original file

In [10]:
questions[81]['question_number'] = '082'

In [12]:
questions[81]

{'question_number': '082',
 'question_summary': 'CONTRIBUINTE DIVORCIADO OU SEPARADO JUDICIALMENTE OU POR ESCRITURA PÚBLICA',
 'question_text': 'Como deve declarar o contribuinte divorciado ou separado judicialmente ou por escritura pública?',
 'answer': ['Apresenta declaração na condição de solteiro, caso não estivesse casado ou vivendo em união estável em',
  '31/12/2023, podendo incluir dependente (nesse caso, devem ser somados os rendimentos recebidos pelo',
  'dependente) do qual detenha a guarda judicial ou deduzir pensão alimentícia paga em face das normas do',
  'direito de família, quando em cumprimento de decisão judicial, inclusive os alimentos provisionais, de acordo',
  'homologado judicialmente ou de escritura pública, a que se refere o art. 733 da Lei nº 13.105, de 16 de março',
  'de 2015 - Código de Processo Civil.',
  'Atenção:',
  'Para efeitos da aplicação da referida dedução, observe-se que:',
  '1) as importâncias pagas relativas ao suprimento de alimentos, em fac

## Format the legal references using LLM

In [44]:
import json

from llm_access import *

import time

### API keys

Expect to have a json file containing all the API keys needed. Groq key should be under "groq" identifier.

In [45]:
API_KEYS_FILE="/work/api_keys_20240427.json"

In [46]:
groq_key = json.load(open(API_KEYS_FILE))['groq']

### Create the Groq model inteface to use

In [47]:
groq_llama33_70b_interface = groq_access(groq_key, GROQ_LLAMA3_3_70B_MODEL)

## Format legal references for all extracted questions

In [48]:
questions[523+36]

{'question_number': '560',
 'question_summary': 'ISENÇÕES DO GANHO DE CAPITAL',
 'question_text': 'Quais são as isenções relativas ao ganho de capital?',
 'answer': ['1 - Indenização da terra nua por desapropriação para fins de reforma agrária, conforme o disposto no § 5º do',
  'art. 184 da Constituição Federal de 1988;',
  'Atenção:',
  'A parcela da indenização, correspondente às benfeitorias, é computada como receita da atividade',
  'rural quando esta tiver sido deduzida como despesa de custeio ou investimento.',
  'A respeito de desapropriação, consulte a pergunta 633',
  '(Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único)',
  '2 - Indenização por liquidação de sinistro, furto ou roubo, relativo ao objeto segurado;',
  '(Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único)',
  '3 - Alienação, por valor igual ou inferior a R$ 440.000,00, do único bem imóvel que o titular possua,',
  'individualmente, em condomínio ou em comunhão, independentemente de

In [49]:
processing_start = time.time()

for i, which_question in enumerate(questions[523+36:677]):
    print("\n*****************************************")
    print(f"Processing question {i}\n")
    print("*****************************************\n")

    cleaned_references = "; ".join(which_question['references']).replace("\n", " ")
    
    if len(cleaned_references) > 0:
        result = legal_references_formatting(groq_llama33_70b_interface, cleaned_references)

        which_question['formatted_references'] = result['referências']
    else:
        print(">>> NO LEGAL REFERENCES TO PROCESS")

        which_question['formatted_references'] = None
    
processing_end = time.time()


*****************************************
Processing question 0

*****************************************


Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único; Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único; Lei nº 9.250, de 26 de dezembro de 1995, art. 23; Lei nº 7.713, de 22 de dezembro de 1998, art. 18; Lei nº 7.713, de 22 de dezembro de 1998, art. 18; Solução de Consulta Cosit nº 17, de 20 de abril de 2022; Solução de Consulta Cosit nº 224, de 25 de setembro de 2023; Nota SEI nº 48/2018/CRJ/PGACET/PGFN-MF; Parecer SEI nº 15069/2022/ME; e Instrução Normativa SRF nº 599, de 28 de dezembro de 2005, art. 2º, § 10, inciso III; Lei nº 11.196, de 21 de novembro de 2005, art. 39; Instrução Normativa SRF nº 599, de 28 de dezembro de 2005, art. 2º; Solução de Consulta Cosit nº 70, de 28 de março de 2014; Solução de Consulta Cosit nº 240, de 19 de maio de 2017; e Solução de Consulta Cosit nº 211, de 24 de junho de 2019; Lei nº 9.250, de 26 de dezembro de 19

In [50]:
print(f"Total time to process the legal references: {processing_end - processing_start}")

Total time to process the legal references: 100.14997053146362


In [51]:
questions[-1]

{'question_number': '715',
 'question_summary': 'ALIENAÇÃO DE AÇÕES EM BOLSA - LIQUIDAÇÃO NO MÊS SUBSEQUENTE',
 'question_text': 'No caso de alienação de ações em pregão ao final de determinado mês, que resulte em momento do recolhimento do imposto?',
 'answer': ['Sendo o ganho líquido sobre renda variável uma modalidade de ganho de capital, a sua tributação segue as',
  'mesmas normas de apuração e tributação do ganho de capital.',
  'Assim, no caso de alienação de ações em bolsa de valores,  tendo em vista que a liquidação financeira não',
  'ocorre na mesma data da operação, o fato gerador do imposto ocorrerá na data do pregão, sendo a tributação',
  'diferida para o momento da liquidação financeira.',
  'Desse modo, para efeitos de apuração do limite de isenção, considera-se a data do fato gerador (data do',
  'pregão). A data da liquidação servirá como parâmetro para a retenção do imposto sobre a renda retido na',
  'fonte (pela corretora) e para a contagem do prazo para recolhime

In [52]:
import pickle

In [53]:
with open("data/extracted_questions_20250207_final.pkl", "wb") as output_file:
    pickle.dump(questions, output_file, pickle.HIGHEST_PROTOCOL)